# Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
funcs = os.listdir('functions')

for func in funcs:
    if func.startswith('__') or func == 'import_all_functions.py':
        continue
    func = func.replace('.py','')
    print(f'{func} imported')
    exec(f'from functions.{func} import *')

import_to_df imported
get_kaggle_data imported
test imported


In [5]:
import pandas as pd 

In [6]:
files = get_kaggle_data(competition="titanic")

files downloaded for titanic competition
   test.csv
   train.csv
   gender_submission.csv
located in /competition_data/


In [7]:
input_data = import_to_df(files)

In [14]:
train = input_data['train']
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [15]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


Missing values in Age column

In [19]:
display(len(train[train['Age'].isna()]))
display(len(train[train['Age'].isna()]) / len(train))

177

0.19865319865319866

Nearly 20% of ages missing. Lets impute those using the mean of Pclass as gender

In [46]:
def impute_cond_mean(
    df: pd.DataFrame,
    col_with_na: str,
    cond_cols: list
) -> pd.DataFrame:
    means_df = (
        df.groupby(cond_cols)
        [col_with_na]
        .mean()
        .reset_index()
    )
    merged_df = df.merge(
        means_df,
        on=cond_cols,
        suffixes=[None, '_imp']
    )
    merged_df[col_with_na].fillna(merged_df[col_with_na + '_imp'], inplace=True)
    merged_df.drop(columns='col_with_na + '_imp', inplace=True)
    return merged_df

In [47]:
test_df = impute_cond_mean(
    df=train,
    col_with_na='Age',
    cond_cols=['Pclass', 'Sex']
)

In [48]:
test_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,A/5 21171,7.2500,NaN,S
1,5,0,3,"Allen, Mr. William Henry",male,35.000000,0,0,373450,8.0500,NaN,S
2,6,0,3,"Moran, Mr. James",male,26.507589,0,0,330877,8.4583,NaN,Q
3,8,0,3,"Palsson, Master. Gosta Leonard",male,2.000000,3,1,349909,21.0750,NaN,S
4,13,0,3,"Saundercock, Mr. William Henry",male,20.000000,0,0,A/5. 2151,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,849,0,2,"Harper, Rev. John",male,28.000000,0,1,248727,33.0000,NaN,S
887,862,0,2,"Giles, Mr. Frederick Edward",male,21.000000,1,0,28134,11.5000,NaN,S
888,865,0,2,"Gill, Mr. John William",male,24.000000,0,0,233866,13.0000,NaN,S
889,884,0,2,"Banfield, Mr. Frederick James",male,28.000000,0,0,C.A./SOTON 34068,10.5000,NaN,S
